In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks
import math

# Load the results
results = np.load('gibbs_results.npy', allow_pickle=True).item()

# ------------------------
# Plotting + Analysis
# ------------------------

n = len(results)
n_cols = 6
n_rows = math.ceil(n / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 2.5 * n_rows), sharex=True, sharey=True)
axes = axes.flatten()
x_vals = np.linspace(-1, 1, 300)

for i, ((beta_h, beta_a), overlaps) in enumerate(results.items()):
    ax = axes[i]
    data = np.array(overlaps)

    # KDE
    kde = gaussian_kde(data)
    kde_vals = kde(x_vals)

    # Multimodality detection
    peaks, _ = find_peaks(kde_vals, prominence=0.01)
    num_peaks = len(peaks)
    modality = 'Multimodal' if num_peaks >= 2 else 'Unimodal'

    # Normality test
    if len(data) >= 3:
        _, pval = shapiro(data)
        normality = 'Normal' if pval > 0.05 else 'Not Normal'
    else:
        normality = 'Too little data'

    # Color scheme
    if modality == 'Multimodal' or normality == 'Not Normal':
        color = 'blue'
    else:
        color = 'green'

    # Plot histogram
    ax.hist(data, bins=20, color=color, edgecolor='black', alpha=0.6, density=True)

    # Plot KDE
    ax.plot(x_vals, kde_vals, color='black', linewidth=2)
    ax.plot(x_vals[peaks], kde_vals[peaks], "ro", markersize=3)

    # Annotate
    ax.set_title(f'β_h={beta_h}, β_AI={beta_a}\n{modality}, {normality}', fontsize=9)
    ax.set_xlim(-1, 1)
    ax.set_yticks([])

for j in range(i + 1, len(axes)):
    axes[j].axis('off')

fig.text(0.5, 0.04, 'Overlap between F_human and F_AI', ha='center', fontsize=12)
plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()
